In [ ]:
!pip install transformers datasets accelerate scikit-learn -q

In [ ]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

print(f"GPU kullanılabilir: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

GPU kullanılabilir: True
GPU: NVIDIA A100-SXM4-80GB


In [ ]:
FOLD_DIR = "/content/drive/MyDrive/nlpveri/filtered_512"
# Sonuçlar buraya kaydedilecek
OUT_ROOT = "/content/drive/MyDrive/berturk_son03"

os.makedirs(OUT_ROOT, exist_ok=True)

# Fold dosyalarını kontrol et
print("Fold dosyaları kontrol ediliyor...")
for fold in range(1, 6):
    train_file = os.path.join(FOLD_DIR, f"train_fold{fold}.jsonl")
    val_file = os.path.join(FOLD_DIR, f"val_fold{fold}.jsonl")

    if os.path.exists(train_file):
        print(f"✅ Fold {fold} train: {train_file}")
    else:
        print(f"❌ Fold {fold} train BULUNAMADI: {train_file}")

    if os.path.exists(val_file):
        print(f"✅ Fold {fold} val: {val_file}")
    else:
        print(f"❌ Fold {fold} val BULUNAMADI: {val_file}")


Fold dosyaları kontrol ediliyor...
✅ Fold 1 train: /content/drive/MyDrive/nlpveri/filtered_512/train_fold1.jsonl
✅ Fold 1 val: /content/drive/MyDrive/nlpveri/filtered_512/val_fold1.jsonl
✅ Fold 2 train: /content/drive/MyDrive/nlpveri/filtered_512/train_fold2.jsonl
✅ Fold 2 val: /content/drive/MyDrive/nlpveri/filtered_512/val_fold2.jsonl
✅ Fold 3 train: /content/drive/MyDrive/nlpveri/filtered_512/train_fold3.jsonl
✅ Fold 3 val: /content/drive/MyDrive/nlpveri/filtered_512/val_fold3.jsonl
✅ Fold 4 train: /content/drive/MyDrive/nlpveri/filtered_512/train_fold4.jsonl
✅ Fold 4 val: /content/drive/MyDrive/nlpveri/filtered_512/val_fold4.jsonl
✅ Fold 5 train: /content/drive/MyDrive/nlpveri/filtered_512/train_fold5.jsonl
✅ Fold 5 val: /content/drive/MyDrive/nlpveri/filtered_512/val_fold5.jsonl


In [ ]:
MODEL_NAME = "dbmdz/bert-base-turkish-cased"
MAX_LEN = 256
NUM_ASPECTS = 25
NUM_CLASSES = 22  # 0..21
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 2e-5
N_FOLDS = 5

# Focal Loss parametreleri
FOCAL_ALPHA = 1.0
FOCAL_GAMMA = 3.0
COMBINED_ALPHA = 0.8  # 0.7 Focal + 0.3 Dice

print("Model Parametreleri:")
print(f"  Model: {MODEL_NAME}")
print(f"  Max Length: {MAX_LEN}")
print(f"  Aspects: {NUM_ASPECTS}")
print(f"  Classes per aspect: {NUM_CLASSES}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Focal Gamma: {FOCAL_GAMMA}")
print(f"  Combined Alpha: {COMBINED_ALPHA}")

Model Parametreleri:
  Model: dbmdz/bert-base-turkish-cased
  Max Length: 256
  Aspects: 25
  Classes per aspect: 22
  Batch Size: 16
  Epochs: 5
  Learning Rate: 2e-05
  Focal Gamma: 3.0
  Combined Alpha: 0.8


In [ ]:
class FocalLoss(nn.Module):
    """
    Focal Loss for multi-class classification
    Zor örneklere odaklanır
    """
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        p_t = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - p_t) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [ ]:
class MultiClassDiceLoss(nn.Module):
    """
    Dice Loss for multi-class classification
    Imbalanced dataset'ler için etkili
    """
    def __init__(self, smooth=1.0):
        super().__init__()
        self.smooth = smooth

    def forward(self, inputs, targets):
        num_classes = inputs.shape[1]
        probs = F.softmax(inputs, dim=1)
        targets_one_hot = F.one_hot(targets, num_classes).float()

        intersection = (probs * targets_one_hot).sum(dim=0)
        cardinality = probs.sum(dim=0) + targets_one_hot.sum(dim=0)

        dice = (2. * intersection + self.smooth) / (cardinality + self.smooth)
        return 1 - dice.mean()

In [ ]:
class CombinedLoss(nn.Module):
    """
    Combined Focal Loss + Dice Loss
    """
    def __init__(self, alpha=0.7, focal_gamma=2.0, focal_alpha=1.0):
        super().__init__()
        self.alpha = alpha
        self.focal_loss = FocalLoss(alpha=focal_alpha, gamma=focal_gamma)
        self.dice_loss = MultiClassDiceLoss()

    def forward(self, inputs, targets):
        focal = self.focal_loss(inputs, targets)
        dice = self.dice_loss(inputs, targets)
        return self.alpha * focal + (1 - self.alpha) * dice

In [ ]:
class BertMultiHeadFocal(nn.Module):
    """
    BERTurk with multiple classification heads + Focal Loss
    """
    def __init__(self, base_model_name: str, num_aspects=25, num_classes=22,
                 dropout=0.3, use_combined_loss=True, focal_gamma=2.0,
                 focal_alpha=1.0, combined_alpha=0.7):
        super().__init__()
        self.bert = AutoModel.from_pretrained(base_model_name)
        hidden = self.bert.config.hidden_size

        self.dropout = nn.Dropout(dropout)
        self.heads = nn.ModuleList([
            nn.Linear(hidden, num_classes) for _ in range(num_aspects)
        ])

        if use_combined_loss:
            self.loss_fn = CombinedLoss(
                alpha=combined_alpha,
                focal_gamma=focal_gamma,
                focal_alpha=focal_alpha
            )
            print(f"Using Combined Loss (alpha={combined_alpha}, gamma={focal_gamma})")
        else:
            self.loss_fn = FocalLoss(alpha=focal_alpha, gamma=focal_gamma)
            print(f"Using Focal Loss (gamma={focal_gamma})")

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls = out.last_hidden_state[:, 0, :]
        cls = self.dropout(cls)

        logits = torch.stack([head(cls) for head in self.heads], dim=1)  # [B, 25, 22]

        loss = None
        if labels is not None:
            labels = labels.long()
            loss_sum = 0.0

            for i in range(len(self.heads)):
                aspect_logits = logits[:, i, :]
                aspect_labels = labels[:, i]
                loss_sum += self.loss_fn(aspect_logits, aspect_labels)

            loss = loss_sum / len(self.heads)

        return {"loss": loss, "logits": logits}


In [ ]:
#ÇALIŞTIRILDI DAHA ÇALIŞTIRMAYIN

import json
import numpy as np

MAX_TOKEN_LENGTH = 256
FILTERED_DIR = os.path.join(FOLD_DIR, "filtered_512")

print("="*60)
print(f"FOLD FİLTRELEME: <= {MAX_TOKEN_LENGTH} TOKENS")
print("="*60)

os.makedirs(FILTERED_DIR, exist_ok=True)

total_original = 0
total_filtered = 0
total_removed = 0

# Her fold için
for fold in range(1, N_FOLDS + 1):
    print(f"\n{'='*20} FOLD {fold} {'='*20}")

    # Dosya yolları
    train_in = os.path.join(FOLD_DIR, f"train_fold{fold}.jsonl")
    val_in = os.path.join(FOLD_DIR, f"val_fold{fold}.jsonl")

    train_out = os.path.join(FILTERED_DIR, f"train_fold{fold}.jsonl")
    val_out = os.path.join(FILTERED_DIR, f"val_fold{fold}.jsonl")

    # Train filtreleme
    print(f"\nTrain filtreleniyor...")
    with open(train_in, 'r', encoding='utf-8') as f:
        train_records = [json.loads(line) for line in f]

    train_filtered = []
    train_removed = 0

    for i, record in enumerate(train_records):
        if i % 2000 == 0:
            print(f"  {i}/{len(train_records)}", end='\r')

        tokens = tokenizer(record['yorum'], truncation=False, add_special_tokens=True)
        length = len(tokens['input_ids'])

        if length <= MAX_TOKEN_LENGTH:
            train_filtered.append(record)
        else:
            train_removed += 1

    print(f"  {len(train_records)}/{len(train_records)} ✅")

    # Val filtreleme
    print(f"Val filtreleniyor...")
    with open(val_in, 'r', encoding='utf-8') as f:
        val_records = [json.loads(line) for line in f]

    val_filtered = []
    val_removed = 0

    for i, record in enumerate(val_records):
        if i % 2000 == 0:
            print(f"  {i}/{len(val_records)}", end='\r')

        tokens = tokenizer(record['yorum'], truncation=False, add_special_tokens=True)
        length = len(tokens['input_ids'])

        if length <= MAX_TOKEN_LENGTH:
            val_filtered.append(record)
        else:
            val_removed += 1

    print(f"  {len(val_records)}/{len(val_records)} ✅")

    # Kaydet
    with open(train_out, 'w', encoding='utf-8') as f:
        for record in train_filtered:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

    with open(val_out, 'w', encoding='utf-8') as f:
        for record in val_filtered:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

    # İstatistikler
    fold_original = len(train_records) + len(val_records)
    fold_filtered = len(train_filtered) + len(val_filtered)
    fold_removed = train_removed + val_removed

    total_original += fold_original
    total_filtered += fold_filtered
    total_removed += fold_removed

    print(f"\nFold {fold} sonuç:")
    print(f"  Train: {len(train_records)} → {len(train_filtered)} (-{train_removed})")
    print(f"  Val:   {len(val_records)} → {len(val_filtered)} (-{val_removed})")
    print(f"  Total: {fold_original} → {fold_filtered} (-{fold_removed}, %{fold_removed/fold_original*100:.1f})")

    print(f"✅ Kaydedildi:")
    print(f"  {train_out}")
    print(f"  {val_out}")

# Genel özet
print("\n" + "="*60)
print("GENEL ÖZET")
print("="*60)
print(f"Toplam orijinal: {total_original} örnek")
print(f"Toplam filtreli: {total_filtered} örnek")
print(f"Toplam silinen:  {total_removed} örnek (%{total_removed/total_original*100:.1f})")
print(f"\n✅ Filtrelenmiş fold'lar: {FILTERED_DIR}")
print(f"\n📌 ARTIK KULLAN:")
print(f'   FOLD_DIR = "{FILTERED_DIR}"')
print("="*60)

FOLD FİLTRELEME: <= 256 TOKENS

==================== FOLD 1 ====================

Train filtreleniyor...
  20881/20881 ✅
Val filtreleniyor...
  5221/5221 ✅

Fold 1 sonuç:
  Train: 20881 → 20446 (-435)
  Val:   5221 → 5100 (-121)
  Total: 26102 → 25546 (-556, %2.1)
✅ Kaydedildi:
  /content/drive/MyDrive/nlpveri/filtered_512/train_fold1.jsonl
  /content/drive/MyDrive/nlpveri/filtered_512/val_fold1.jsonl

==================== FOLD 2 ====================

Train filtreleniyor...
  20882/20882 ✅
Val filtreleniyor...
  5220/5220 ✅

Fold 2 sonuç:
  Train: 20882 → 20428 (-454)
  Val:   5220 → 5118 (-102)
  Total: 26102 → 25546 (-556, %2.1)
✅ Kaydedildi:
  /content/drive/MyDrive/nlpveri/filtered_512/train_fold2.jsonl
  /content/drive/MyDrive/nlpveri/filtered_512/val_fold2.jsonl

==================== FOLD 3 ====================

Train filtreleniyor...
  20882/20882 ✅
Val filtreleniyor...
  5220/5220 ✅

Fold 3 sonuç:
  Train: 20882 → 20430 (-452)
  Val:   5220 → 5116 (-104)
  Total: 26102 → 25546 

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(batch):
    texts = batch["yorum"]
    enc = tokenizer(texts, truncation=True, padding=False, max_length=MAX_LEN)  # padding yok, collator pad’leyecek
    enc["labels"] = batch["labels"]
    return enc

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # <-- EKLENDİ

print("Tokenizer + data_collator hazır.")


Tokenizer + data_collator hazır.


In [ ]:
def run_cv_training():
    """
    HAZIR fold'ları kullanarak model eğitimi
    jsonolusturma.py tarafından oluşturulmuş fold'lar
    """
    results = []

    for fold in range(1, N_FOLDS + 1):
        print(f"\n{'='*20} FOLD {fold} {'='*20}")

        # HAZIR FOLD'LARI YÜK
        train_path = os.path.join(FOLD_DIR, f"train_fold{fold}.jsonl")
        val_path = os.path.join(FOLD_DIR, f"val_fold{fold}.jsonl")

        if not os.path.exists(train_path):
            raise FileNotFoundError(f"Fold dosyası bulunamadı: {train_path}")
        if not os.path.exists(val_path):
            raise FileNotFoundError(f"Fold dosyası bulunamadı: {val_path}")

        print(f"Loading fold data:")
        print(f"  Train: {train_path}")
        print(f"  Val: {val_path}")

        # Veriyi yükle
        train_data = load_dataset("json", data_files=train_path, split="train")
        val_data = load_dataset("json", data_files=val_path, split="train")

        print(f"Train: {len(train_data)} samples")
        print(f"Val: {len(val_data)} samples")

        # Tokenize
        train_tokenized = train_data.map(
            preprocess,
            batched=True,
            remove_columns=train_data.column_names
        )
        val_tokenized = val_data.map(
            preprocess,
            batched=True,
            remove_columns=val_data.column_names
        )

        # Yeni model oluştur (HER FOLD İÇİN YENİ MODEL)
        model = BertMultiHeadFocal(
            MODEL_NAME,
            num_aspects=NUM_ASPECTS,
            num_classes=NUM_CLASSES,
            dropout=0.1,
            use_combined_loss=True,
            focal_gamma=FOCAL_GAMMA,
            combined_alpha=COMBINED_ALPHA
        )

        # Çıktı dizini
        out_dir = f"{OUT_ROOT}/fold{fold}"
        os.makedirs(out_dir, exist_ok=True)

        # Training arguments
        args = TrainingArguments(
            output_dir=out_dir,
            overwrite_output_dir=True,

            num_train_epochs=EPOCHS,
            per_device_train_batch_size=BATCH_SIZE,
            per_device_eval_batch_size=BATCH_SIZE,

            learning_rate=LEARNING_RATE,
            weight_decay=0.05,
            warmup_ratio=0.1,
            label_smoothing_factor=0.1,

            eval_strategy="steps",
            eval_steps=500,
            save_strategy="steps",
            save_steps=500,
            save_total_limit=2,


            load_best_model_at_end=True,
            metric_for_best_model="nonzero_accuracy",
            greater_is_better=True,

            logging_steps=100,
            report_to="none",

            fp16=True,
            dataloader_num_workers=4,
        )

        # Trainer
        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=train_tokenized,
            eval_dataset=val_tokenized,
            compute_metrics=compute_metrics,
            data_collator=data_collator,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        # Eğitim
        print("Eğitim başlıyor...")
        trainer.train()

        # Final evaluation
        print("Final evaluation...")
        eval_result = trainer.evaluate()
        eval_result["fold"] = fold
        results.append(eval_result)

        print(f"Fold {fold} eval: {eval_result}")

        # Modeli kaydet
        trainer.save_model(out_dir)
        print(f"Model kaydedildi: {out_dir}")

    return results


# Eğitimi başlat
cv_results = run_cv_training()


==================== FOLD 1 ====================
Loading fold data:
  Train: /content/drive/MyDrive/nlpveri/filtered_512/train_fold1.jsonl
  Val: /content/drive/MyDrive/nlpveri/filtered_512/val_fold1.jsonl
Train: 20446 samples
Val: 5100 samples


Map:   0%|          | 0/5100 [00:00<?, ? examples/s]

Using Combined Loss (alpha=0.8, gamma=3.0)
Eğitim başlıyor...


Step,Training Loss,Validation Loss,Accuracy Mean,Nonzero Accuracy,Aspect Presence F1,Presence Precision,Presence Recall
500,0.159100,0.133028,0.907239,0.440395,0.607101,0.813739,0.484156
1000,0.089500,0.080368,0.941310,0.663454,0.826950,0.897426,0.766737
1500,0.069900,0.066342,0.949396,0.732328,0.866397,0.895373,0.839238
2000,0.062700,0.060525,0.952463,0.758404,0.879862,0.894604,0.865597
2500,0.061800,0.056627,0.955153,0.772292,0.884641,0.900188,0.869622
3000,0.051800,0.053696,0.956047,0.774899,0.888210,0.904116,0.872853
3500,0.052600,0.053393,0.955122,0.801372,0.892446,0.878266,0.907091
4000,0.041100,0.050581,0.957733,0.791622,0.894815,0.901455,0.888272
4500,0.042000,0.049994,0.957451,0.802506,0.896633,0.891285,0.902046
5000,0.042000,0.048988,0.958227,0.802619,0.898063,0.895960,0.900176


Final evaluation...


...

Map:   0%|          | 0/5117 [00:00<?, ? examples/s]

Using Combined Loss (alpha=0.8, gamma=3.0)
Eğitim başlıyor...


Step,Training Loss,Validation Loss,Accuracy Mean,Nonzero Accuracy,Aspect Presence F1,Presence Precision,Presence Recall
500,0.156500,0.130804,0.915763,0.474317,0.658803,0.869073,0.530460
1000,0.089900,0.082571,0.939848,0.646791,0.820067,0.904452,0.750084
1500,0.075600,0.069938,0.945812,0.714005,0.857184,0.888788,0.827751
2000,0.067300,0.064276,0.950033,0.722659,0.869649,0.910438,0.832359
2500,0.061600,0.061698,0.950877,0.757503,0.880505,0.887569,0.873553
3000,0.054700,0.057730,0.953105,0.772564,0.883681,0.888984,0.878442
3500,0.049600,0.055675,0.953653,0.768855,0.885415,0.895596,0.875464
4000,0.046000,0.054272,0.954380,0.771608,0.888611,0.898254,0.879173
4500,0.043400,0.053036,0.955708,0.771159,0.889576,0.906925,0.872878


Final evaluation...


Fold 5 eval: {'eval_loss': 0.05773007497191429, 'eval_accuracy_mean': 0.9531053351573188, 'eval_nonzero_accuracy': 0.7725637855456896, 'eval_aspect_presence_f1': 0.8836814874723792, 'eval_presence_precision': 0.888983677415635, 'eval_presence_recall': 0.87844217151844, 'eval_runtime': 9.8825, 'eval_samples_per_second': 517.782, 'eval_steps_per_second': 32.38, 'epoch': 3.523884103367267, 'fold': 5}
Model kaydedildi: /content/drive/MyDrive/berturk_focal_cv/fold5


In [ ]:
def summarize_results(results):
    """
    Cross-validation sonuçlarını özetle
    """
    metrics = ["eval_loss", "eval_accuracy_mean", "eval_nonzero_accuracy",
               "eval_aspect_presence_f1", "eval_presence_precision", "eval_presence_recall"]

    summary = {}
    for metric in metrics:
        values = [r[metric] for r in results if metric in r]
        summary[metric] = {
            "mean": float(np.mean(values)),
            "std": float(np.std(values, ddof=1)) if len(values) > 1 else 0.0,
            "values": values
        }

    return summary

summary = summarize_results(cv_results)

print("\n" + "="*60)
print("CROSS-VALIDATION SUMMARY (5-fold)")
print("="*60)
for metric, stats in summary.items():
    print(f"{metric}:")
    print(f"  mean = {stats['mean']:.6f}")
    print(f"  std  = {stats['std']:.6f}")

# Sonuçları kaydet
results_file = f"{OUT_ROOT}/cv_results.json"
summary_file = f"{OUT_ROOT}/cv_summary.json"

with open(results_file, "w", encoding="utf-8") as f:
    json.dump(cv_results, f, ensure_ascii=False, indent=2)

with open(summary_file, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print(f"\nSonuçlar kaydedildi:")
print(f"  - {results_file}")
print(f"  - {summary_file}")


old_nonzero = 0.7613
new_nonzero = summary["eval_nonzero_accuracy"]["mean"]
improvement = (new_nonzero - old_nonzero) * 100

print(f"Eski nonzero accuracy: {old_nonzero:.2%}")
print(f"Yeni nonzero accuracy: {new_nonzero:.2%}")
print(f"İyileşme: {improvement:+.2f}%")




CROSS-VALIDATION SUMMARY (5-fold)
eval_loss:
  mean = 0.051458
  std  = 0.003954
eval_accuracy_mean:
  mean = 0.957129
  std  = 0.002333
eval_nonzero_accuracy:
  mean = 0.792112
  std  = 0.011487
eval_aspect_presence_f1:
  mean = 0.896098
  std  = 0.006998
eval_presence_precision:
  mean = 0.898463
  std  = 0.006228
eval_presence_recall:
  mean = 0.893765
  std  = 0.008995

Sonuçlar kaydedildi:
  - /content/drive/MyDrive/berturk_focal_cv/cv_results.json
  - /content/drive/MyDrive/berturk_focal_cv/cv_summary.json
Eski nonzero accuracy: 76.13%
Yeni nonzero accuracy: 79.21%
İyileşme: +3.08%
